# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [402]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [403]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [404]:
# Load the historical closing prices for Bitcoin
df = pd.read_csv('Resources/BTC_USD_2013-10-01_2020-12-29-CoinDesk.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)
df.head()

,Currency,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
Date,,,,,
2013-10-01,BTC,123.65499,124.30466,124.75166,122.56349
2013-10-02,BTC,125.45500,123.65499,125.75850,123.63383
2013-10-03,BTC,108.58483,125.45500,125.66566,83.32833
2013-10-04,BTC,118.67466,108.58483,118.67500,107.05816
2013-10-05,BTC,121.33866,118.67466,121.93633,118.00566


In [406]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [407]:
def get_trailing_window_data(df, num_trailing_windows, window_length, feature_col_number, target_col_number):
    X = []
    y = []

    i_start = len(df) - window_length - num_trailing_windows
    i_end = i_start + num_trailing_windows

    for i in range(i_start, i_end):
        features = df.iloc[i:(i + window_length), feature_col_number]
        target = df.iloc[i + window_length, target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)


In [ ]:
def append_window_data(X, Y,
        df, num_trailing_windows, window_length, feature_col_number, target_col_number):
    X_new, y_new = get_trailing_window_data(df, num_trailing_windows, window_length, feature_col_number, target_col_number)
    print(f"X_new.shape {X_new.shape}  y_new.shape {y_new.shape}")
    return (np.append(X, X_new, axis=0),  np.append(Y, y_new, axis=0))

In [414]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [421]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [426]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.

scaler = MinMaxScaler()
scaler.fit(X_train)
# scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [427]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[7.90736096e-04]
  [8.85183194e-04]
  [0.00000000e+00]
  [0.00000000e+00]
  [3.58772654e-05]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [0.00000000e+00]
  [3.57898734e-05]]

 [[8.85183194e-04]
  [0.00000000e+00]
  [5.29416594e-04]
  [1.39854970e-04]
  [0.00000000e+00]
  [5.98367452e-05]
  [6.50033109e-05]
  [5.33502885e-05]
  [1.00413121e-04]
  [0.00000000e+00]]

 [[0.00000000e+00]
  [5.29416594e-04]
  [6.69197523e-04]
  [1.03981435e-04]
  [5.98367452e-05]
  [1.24836166e-04]
  [1.18350131e-04]
  [1.53758052e-04]
  [6.46255601e-05]
  [3.40221740e-05]]

 [[5.29416594e-04]
  [6.69197523e-04]
  [6.33342980e-04]
  [1.63811959e-04]
  [1.24836166e-04]
  [1.78179795e-04]
  [2.18751368e-04]
  [1.17972401e-04]
  [9.86455354e-05]
  [5.79606891e-05]]

 [[6.69197523e-04]
  [6.33342980e-04]
  [6.93141828e-04]
  [2.28804621e-04]
  [1.78179795e-04]
  [2.78575024e-04]
  [1.82968043e-04]
  [1.51990561e-04]
  [1.22582503e-04]
  [5.23184930e-04]]] 

X_test sample v

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:

1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [428]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [429]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()

number_units = 5
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))


In [430]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [431]:
# Summarize the model
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 10, 5)             140       
_________________________________________________________________
dropout_21 (Dropout)         (None, 10, 5)             0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 10, 5)             220       
_________________________________________________________________
dropout_22 (Dropout)         (None, 10, 5)             0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 5)                 220       
_________________________________________________________________
dropout_23 (Dropout)         (None, 5)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [432]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
1839/1839 [==============================] - 8s 4ms/step - loss: 7.2636e-04A: 0s - loss
Epoch 2/10
1839/1839 [==============================] - 8s 4ms/step - loss: 0.0017
Epoch 3/10
1839/1839 [==============================] - 8s 4ms/step - loss: 0.0017
Epoch 4/10
1106/1839 [=================>............] - ETA: 3s - loss: 0.0010

KeyboardInterrupt: 

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [ ]:
# Evaluate the model
model.evaluate(X_test, y_test)

In [ ]:
# Loop to build each day's prediction.
predicted = model.predict(X_test)

current_day = pd.to_datetime( datetime.now().strftime("%Y-%m-%d") )
num_days = 20

for i inrange(1, num_days + 1):


X, y = append_window_data(X, y, df, num_trailing_windows=1, window_length=10, feature_col_number=1, target_col_number=1)


# # Append days in the future
# from datetime import datetime
# from datetime import timedelta

# current_day = pd.to_datetime( datetime.now().strftime("%Y-%m-%d") )
# num_days = 20

# for i in range(1, num_days + 1):
#     df.loc[current_day + timedelta(days=i)] = ["BTC", 0, 0, 0, 0]

# df.tail()


In [ ]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

In [ ]:
# Plot the real vs predicted values as a line chart
stocks.plot()